In [1]:
!omz_downloader --name face-detection-adas-0001 --precision FP16

################|| Downloading face-detection-adas-0001 ||################

========== Downloading C:\BrainAI\face-detections\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.xml
... 100%, 304 KB, 330 KB/s, 0 seconds passed

========== Downloading C:\BrainAI\face-detections\intel\face-detection-adas-0001\FP16\face-detection-adas-0001.bin
... 49%, 1024 KB, 172 KB/s, 5 seconds passed
... 99%, 2048 KB, 280 KB/s, 7 seconds passed
... 100%, 2056 KB, 281 KB/s, 7 seconds passed



In [15]:
import openvino as ov
import cv2
import numpy as np

import matplotlib.pyplot as plt
from pathlib import Path

In [16]:
core = ov.Core()
options=core.available_devices

options

['CPU', 'GPU']

In [17]:
image = cv2.imread("data/test.jpg")
resized_image = cv2.resize(src=image, dsize=(672, 384)) 
transposed_image = resized_image.transpose(2, 0, 1)
input_image = np.expand_dims(transposed_image, 0)

In [18]:
import openvino as ov

In [19]:
core = ov.Core()

model = core.read_model(model="models/face-detection-adas-0001.xml")
face_model = core.compile_model(model=model, device_name="CPU")

face_input_layer = face_model.input(0)
face_output_layer = face_model.output(0)
print("Input layer shape: ", face_input_layer.shape)
print("Output layer shape: ", face_output_layer.shape)

Input layer shape:  [1,3,384,672]
Output layer shape:  [1,1,200,7]


In [20]:
face_output = face_model([input_image])[face_output_layer]

In [21]:
def DrawBoundingBoxes(output, image, conf):
    canvas = image.copy()
    h,w,_ = canvas.shape

    predictions = output[0][0]
    confidence = predictions[:,2]
    top_predictions = predictions[(confidence>conf)]

    for detection in top_predictions:
        box = detection[3:7] * np.array([w, h, w, h])
        (xmin, ymin, xmax, ymax) = box.astype("int")
        cv2.rectangle(canvas, (xmin, ymin), (xmax, ymax), (0,0,255),2)

    return canvas

In [22]:
def AddBackground(frame, bg):

    frame_h, frame_w = frame.shape[0], frame.shape[1]
    new_h = 500
    new_w = int((new_h/frame_h)*frame_w)
    frame_resize = cv2.resize(frame, (new_w, new_h))

    xmax = bg.shape[1] - 350
    ymax = bg.shape[0] - 175
    xmin = xmax - new_w
    ymin = ymax - new_h

    bg[ymin:ymax, xmin:xmax] = frame_resize

    return bg

In [29]:
camera = cv2.VideoCapture(0) #create a VideoCapture object with the 'first' camera (your webcam)
background = "./data/Background.jpg"
bg = cv2.imread(background)

while(True):
    ret, frame = camera.read()             # Capture frame by frame      
    if ret == False:
        break
    
    resized_frame = cv2.resize(src=frame, dsize=(672, 384)) 
    transposed_frame = resized_frame.transpose(2, 0, 1)
    input_frame = np.expand_dims(transposed_frame, 0)    
    
    face_output = face_model([input_frame])[face_output_layer]

    canvas = DrawBoundingBoxes(face_output, frame, conf=0.5)
    deployment = AddBackground(canvas, bg)
    
    cv2.imshow('Press Spacebar to Exit', deployment)

    if cv2.waitKey(1) & 0xFF == ord(' '):  # Stop if spacebar is detected
        break

camera.release()                           # Cleanup after spacebar is detected.
cv2.destroyAllWindows()